<a href="https://colab.research.google.com/github/suyash091/Cardiovascular-Disease-Prediction/blob/master/Cardiovascular%20Disease%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**IMPORTING THE DATASET**


In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "suyash091"
os.environ['KAGGLE_KEY'] = "9775b0a6da90b12a0c284d71e498de43"
!kaggle datasets download -d sulianova/cardiovascular-disease-dataset

  0% 0.00/742k [00:00<?, ?B/s]
100% 742k/742k [00:00<00:00, 46.7MB/s]


**Extracting zipfile**

In [0]:
!unzip -q '/content/cardiovascular-disease-dataset.zip' 

In [0]:
import pandas as pd

**Data Preprocessing**

In [0]:
#Fixing structure
# Read in the file
with open('/content/cardio_train.csv', 'r') as file :
  filedata = file.read()

# Replace the target string
filedata = filedata.replace(';', ',')

# Write the file out again
with open('/content/cardio_train.csv', 'w') as file:
  file.write(filedata)

In [44]:
#Load Data
cvd=pd.read_csv('/content/cardio_train.csv')#.apply(lambda x: x/x.max(), axis=0)
cvd.describe()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,19468.865814,1.349571,164.359229,74.205690,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,2467.251667,0.476838,8.210126,14.395757,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,10798.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,17664.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,19703.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,21327.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,23713.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
cvd.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [0]:
model = RandomForestClassifier(n_estimators=100)
outcome_var = ['cardio']
predictor_var = ['age', 'gender', 'height', 'weight','ap_hi','ap_lo', 'cholesterol','gluc','smoke',	'alco',	'active']

In [0]:
#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
import numpy as np

#Generic function for making a classification model and accessing performance:
def classification_model(model, data, predictors, outcome):
  model.fit(data[predictors],data[outcome].values.ravel())
  
  predictions = model.predict(data[predictors])
  
  accuracy = metrics.accuracy_score(predictions,data[outcome])
  print('Accuracy : %s' % '{0:.3%}'.format(accuracy))

  kf = KFold(n_splits=5)
  error = []
  for train, test in kf.split(data[predictors]):
    train_predictors = (data[predictors].iloc[train,:])
    
    train_target = data[outcome].iloc[train]
    
    model.fit(train_predictors, train_target.values.ravel())
    
    error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
 
  print('Cross-Validation Score : %s' % '{0:.3%}'.format(np.mean(error)))

  return model.fit(data[predictors],data[outcome].values.ravel())


In [47]:

#outcome_var = ['winner']
#predictor_var = ['team1', 'team2', 'venue', 'toss_winner','city','toss_decision']
history=classification_model(model, cvd,predictor_var,outcome_var)

Accuracy : 99.970%
Cross-Validation Score : 71.461%


In [0]:
#inp=list(map(str,[18393,	2,	168,	62.0,	110,	80,	1,	1,	0,	0,	1]))
inp=list(map(str,'20228	1	156	85.0	140	90	3	1	0	0	1'.split()))
inp = np.array(inp).reshape((1, -1))
output=model.predict(inp)

In [53]:
print(output)

[1]


In [54]:
from sklearn.externals import joblib
joblib.dump(cvd, 'Cardiomodel.pkl')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['Cardiomodel.pkl']